In [32]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier


In [33]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

train.head()

,Loan_ID,Loan_Amount_Requested,Loan_Grade,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Area_Type,Gender,Interest_Rate
0,10000001,"7,000",A,< 1 year,Rent,68000.0,not verified,car,18.37,0,NaN,9,14,B,Female,1
1,10000002,"30,000",A,4 years,Mortgage,NaN,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,A,Female,3
2,10000003,"24,725",D,7 years,Mortgage,75566.4,VERIFIED - income source,debt_consolidation,15.88,0,NaN,12,16,B,Male,3
3,10000004,"16,000",D,< 1 year,NaN,56160.0,VERIFIED - income source,debt_consolidation,14.34,3,NaN,16,22,A,Male,3
4,10000005,"17,000",A,8 years,Own,96000.0,VERIFIED - income source,debt_consolidation,22.17,1,NaN,19,30,B,Female,1


In [34]:
train=train.drop('Loan_ID',1)
loan_id=test.Loan_ID
test=test.drop('Loan_ID',1)

In [35]:
#Imputing missing values for both train and test
train.fillna(-999, inplace=True)
test.fillna(-999,inplace=True)


#Creating a training set for modeling and validation set to check model performance
y = train.Interest_Rate
X = train.drop(['Interest_Rate'], axis=1)


from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.7, random_state=1234)

In [54]:
categorical_features_indices = np.where(X.dtypes != np.float)[0]
#importing library and building model

model=CatBoostClassifier(iterations=50, depth=5, learning_rate=0.1,loss_function='MultiClass')


In [55]:
model.fit(X_train, y_train,cat_features=categorical_features_indices,eval_set=(X_validation, y_validation),plot=True)

In [56]:
submission = pd.DataFrame()
submission['Loan_ID'] =loan_id
submission['Interest_Rate'] = model.predict(test)
submission.to_csv("Submission.csv")